# Práctica 3 - Recurrent Word2Vectors - Pokémon Legends: Arceus

## Alumno: Espinosa Cruz Emilio Jared

## Introducción

A lo largo del curso se ha podido conocer y trabajar con diferentes tipos de redes neuronales: convolucionales, residuales, y ahora, un nuevo tipo de redes: las recurrentes.

Las redes recurrentes son un tipo de red neuronal artificial que se utilizan para procesar datos secuenciales. Son particularmente adecuadas para tareas como el reconocimiento de voz, la traducción automática y la generación de texto.

Para poder entender un poco más este concepto, además de explorar de nuevo el tema de aprendizaje no supervisado, se implementará un modelo CBOW desde cero.

El modelo CBOW (o bolsa de palabras) es un modelo de red neuronal que permite crear representaciones distribuidas de palabras que capturan las relaciones semánticas y sintácticas entre palabras.

La implementación de este modelo se ajusta a uno de los objetivos de la práctica, ya que este crea diferentes agrupaciones con base al conjunto de datos entrenados. Por lo tanto, podremos ver qué tipo de agrupaciones crea el modelo a lo largo del modelo.

Se dividirá en dos partes: la primera parte será para el preprocesamiento del conjunto de datos, las cuales implicarán aplicar técnicas de limpieza de datos para lenguaje natural, como eliminación de stop words, tokenización y lematización. La segunda parte será la implementación del modelo CBOW, donde se creará el vocabulario completo a partir de los datos, su respectiva tokenización y la implementación y entrenamiento del modelo,
 para después ver los resultados a partir de una proyección de palabras hechas por este.

Una vez descritas las tareas, se empezará con la parte del preprocesamiento del dataset.

In [1]:
import pandas as pd
import spacy
from langdetect import detect
import re

## Cargar el conjunto de datos

El dataset a emplear consiste en una colección de comentarios de diversos idiomas acerca del juego 'Pokémon Legends: Arceus'.

Como primera parte ser cargará el dataset que ese encuentra en formato CSV.

In [2]:
dataframe = pd.read_csv('./arceus_reviews.csv')

Una vez cargado el conjunto de datos, procedemos a explorar las primeras 5 tuplas del dataset para conocer como se encuentra conformado.

In [3]:
dataframe.head()

,review,date,score,helpful,number_of_reviews,number_of_ratings,average_user_score
0,"A fun reinvention of the Pokémon formula, hope...","Jan 29, 2022",10,49 of 62 users found this helpful,2 Reviews,2 Ratings,0.0
1,"A fun, challenging (at times), Pokemon game wi...","Jan 30, 2022",10,52 of 72 users found this helpful,60 Reviews,111 Ratings,5.8
2,El juego más infravalorado de la saga y posibl...,"Jan 29, 2022",10,48 of 62 users found this helpful,8 Reviews,12 Ratings,7.2
3,"Best pokemon game of all time! New pokemon, ne...","Jan 30, 2022",10,28 of 35 users found this helpful,1 Review,4 Ratings,5.0
4,"One outstanding experience, that is certain. I...","Feb 3, 2022",10,4 of 8 users found this helpful,1 Review,14 Ratings,9.0


Explorado el dataset, procedemos a extraer la información que no interesa, las opiniones.

In [4]:
reviews = dataframe['review'].astype('str').tolist()

## Identificar el idioma de la review

Como se mencionó anteriormente, el dataset está conformado por diferentes idiomas, por lo tanto, se iterará cada opinión para saber en que idioma está escrito antes de preprocesarlo.

In [5]:
languajes = set()

In [6]:
languajes_review = []
for review in reviews:
  languaje = detect(review)
  languajes.add(languaje)
  languajes_review.append(languaje)

In [7]:
languajes

{'ca',
 'de',
 'en',
 'es',
 'fr',
 'id',
 'it',
 'ja',
 'ko',
 'no',
 'pt',
 'ru',
 'tr',
 'zh-cn'}

### Lenguajes detectados

Para analizar en que idioma está escrita la opinión se usó la librería *langdetect*, la cual detectó las siguientes librerías.

- 'ca': Catalán
- 'de': Alemán
- 'en': Inglés
- 'es': Español
- 'fr': Francés
- 'id': Indonesio
- 'it': Italiano
- 'ja': Japonés
- 'ko': Coreano
- 'no': Noruego
- 'pt': Portugués
- 'ru': Ruso
- 'tr': Turco
- 'zh-cn': Chino Simplificado

## Preprocesamiento

Una vez descubiertos los idiomas se procederá a procesar las opiones usando la librería *Spacy*. Sin embargo, no se realizará el procesamiento de todas las opiniones, ya que esta librería no tiene soporte para todos los idiomas detectados anteriormente. Por lo tanto, las opiniones antes mencionadas se quedarán de esa manera.

Se crean listas para poder guardar cada opinión respecto a su idioma:

In [10]:
rev_catalan = []
rev_aleman = []
rev_ingles = []
rev_español = []
rev_frances = []
rev_italiano = []
rev_japones = []
rev_coreano = []
rev_noruego = []
rev_portugues = []
rev_ruso = []
rev_chino = []
rev_general = []

In [11]:
for review, languaje in zip(reviews, languajes_review):
  if languaje == 'ca':
    rev_catalan.append(review)
  elif languaje == 'de':
    rev_aleman.append(review)
  elif languaje == 'en':
    rev_ingles.append(review)
  elif languaje == 'es':
    rev_español.append(review)
  elif languaje == 'fr':
    rev_frances.append(review)
  elif languaje == 'it':
    rev_italiano.append(review)
  elif languaje == 'ja':
    rev_japones.append(review)
  elif languaje == 'ko':
    rev_coreano.append(review)
  elif languaje == 'no':
    rev_noruego.append(review)
  elif languaje == 'pt':
    rev_portugues.append(review)
  elif languaje == 'ru':
    rev_ruso.append(review)
  elif languaje == 'zh-cn':
    rev_chino.append(review)
  else:
    rev_general.append(review)

### Aplicar preprocesamiento para cada una de las reviews dependiendo del idioma

Se crea un diccionario con distintas reglas que se aplicarán para limipiar cierto contenido de la review.

In [17]:
clean_dict = {
  'user': r'\B@\w+',
  'hashtag': r'\B#\w+',
  'links': r'\b(?:https?|ftp):\/\/\S+',
  # 'especialWords': r'\b[A-Z]{2,3}\b|&amp;|(?<=\s)(via|Via)(?=\s)',
  'notCharacter': r'[“”",.¿?¡!#$%/():&;^-]|\b\'(?!\w)|(?<!\w)\'\b|…',
  'spaces': r'\s{1,}',
  'start': r'^\s+',
  'consonants': r'\b[b-df-hj-np-tv-zB-DF-HJ-NP-TV-Z]\b'
}

In [18]:
def cleanReview(review):
  aux = re.sub(clean_dict['user'] , '', review)
  aux = re.sub(clean_dict['hashtag'] , '', aux)
  aux = re.sub(clean_dict['links'], '', aux)
  # aux = re.sub(twitter_dict['especialWords'], ' ', aux)
  aux = re.sub(clean_dict['notCharacter'], ' ', aux)
  aux = re.sub(clean_dict['consonants'], ' ', aux)
  aux = re.sub(clean_dict['spaces'], ' ', aux)
  aux = re.sub(clean_dict['start'], '', aux)
  return aux

Definido el diccionario y la función que realizará la acción, se procede a procesar las opiones, donde primero se aplican las reglas del diccionario y después pasan al su respectivo proceso de tokenización y lematización dependiento el idioma. Importante aclarar que, no todas las reviews pasarán por el proceso de limpieza con el diccionario, ya que, muchos manejan un sistemas de escritura diferente:

### Catalán

In [19]:
nlp = spacy.load("ca_core_news_sm")

clean_catalan = []
for review in rev_catalan:
  review = cleanReview(review)
  doc = nlp(review)
  aux = ''
  for token in doc:
    aux = aux + token.lemma_ + ' '
    # print(token.lemma_)
  clean_catalan.append(aux)

### Alemán

In [20]:
nlp = spacy.load("nl_core_news_sm")

clean_aleman = []
for review in rev_aleman:
  review = cleanReview(review)
  doc = nlp(review)
  aux = ''
  for token in doc:
    aux = aux + token.lemma_ + ' '
    # print(token.lemma_)
  clean_aleman.append(aux)

### Inglés

In [21]:
nlp = spacy.load("en_core_web_sm")

clean_ingles = []
for review in rev_ingles:
  review = cleanReview(review)
  doc = nlp(review)
  aux = ''
  for token in doc:
    aux = aux + token.lemma_ + ' '
    # print(token.lemma_)
  clean_ingles.append(aux)

### Español

In [22]:
nlp = spacy.load("es_core_news_sm")

clean_espanol = []
for review in rev_español:
  review = cleanReview(review)
  doc = nlp(review)
  aux = ''
  for token in doc:
    aux = aux + token.lemma_ + ' '
    # print(token.lemma_)
  clean_espanol.append(aux)

### Francés

In [23]:
nlp = spacy.load("fr_core_news_sm")

clean_frances = []
for review in rev_frances:
  review = cleanReview(review)
  doc = nlp(review)
  aux = ''
  for token in doc:
    aux = aux + token.lemma_ + ' '
    # print(token.lemma_)
  clean_frances.append(aux)

### Italiano

In [24]:
nlp = spacy.load("it_core_news_sm")

clean_italiano = []
for review in rev_italiano:
  review = cleanReview(review)
  doc = nlp(review)
  aux = ''
  for token in doc:
    aux = aux + token.lemma_ + ' '
    # print(token.lemma_)
  clean_italiano.append(aux)

### Japonés

In [25]:
nlp = spacy.load("ja_core_news_sm")

clean_japones = []
for review in rev_japones:
  # review = cleanReview(review)
  doc = nlp(review)
  aux = ''
  for token in doc:
    aux = aux + token.lemma_ + ' '
    # print(token.lemma_)
  clean_japones.append(aux)

### Coreano

In [26]:
nlp = spacy.load("ja_core_news_sm")

clean_coreano = []
for review in rev_coreano:
  # review = cleanReview(review)
  doc = nlp(review)
  aux = ''
  for token in doc:
    aux = aux + token.lemma_ + ' '
    # print(token.lemma_)
  clean_coreano.append(aux)

### Noruego

In [27]:
nlp = spacy.load("nb_core_news_sm")

clean_noruego = []
for review in rev_noruego:
  # review = cleanReview(review)
  doc = nlp(review)
  aux = ''
  for token in doc:
    aux = aux + token.lemma_ + ' '
    # print(token.lemma_)
  clean_noruego.append(aux)

### Ruso

In [28]:
nlp = spacy.load("ru_core_news_sm")

clean_ruso = []
for review in rev_ruso:
  # review = cleanReview(review)
  doc = nlp(review)
  aux = ''
  for token in doc:
    aux = aux + token.lemma_ + ' '
    # print(token.lemma_)
  clean_ruso.append(aux)

### Chino

In [29]:
nlp = spacy.load("zh_core_web_sm")

clean_chino = []
for review in rev_chino:
  # review = cleanReview(review)
  doc = nlp(review)
  aux = ''
  for token in doc:
    aux = aux + token.lemma_ + ' '
    # print(token.lemma_)
  clean_chino.append(aux)

### Portugues

In [30]:
nlp = spacy.load("pt_core_news_sm")

clean_portuges = []
for review in rev_portugues:
  # review = cleanReview(review)
  doc = nlp(review)
  aux = ''
  for token in doc:
    aux = aux + token.lemma_ + ' '
    # print(token.lemma_)
  clean_portuges.append(aux)

## Crear dataframe final para guardar

Finalizadas todas las tareas, se procede a crear un dataframe con todas las opiniones finales, el cual se guardará en un archivo CSV para la siguiente parte de la práctica.

In [32]:
final_df = pd.DataFrame({
  'reviews': clean_aleman + clean_catalan + clean_chino + clean_coreano + clean_espanol + clean_frances + rev_general + clean_ingles + clean_italiano + clean_japones + clean_noruego + clean_portuges + clean_ruso,

})
final_df.to_csv('dataset_review.csv', index=False)

En la siguiente parte se realizarán las tareas restantes, como la creación del modelo y su entrenamiento, al igual que su visualización.